In [109]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
#!pip install tensorflow-transform
#import tensorflow_transform as tft

# Modelamiento

Train - test split

In [105]:
xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

x_train, x_test, y_train, y_test = train_test_split(xs, ys, test_size=0.2)

# Normalización de los datos de entrada
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train.reshape(-1, 1))
x_test = scaler.transform(x_test.reshape(-1, 1))

# Elementos del Modelo

In [110]:
# Métricas:
METRICS = [
    keras.metrics.TruePositives(name='tp'),  # Verdaderos positivos
    keras.metrics.FalsePositives(name='fp'),  # Falsos positivos
    keras.metrics.TrueNegatives(name='tn'),  # Verdaderos negativos
    keras.metrics.FalseNegatives(name='fn'),  # Falsos negativos
    keras.metrics.BinaryAccuracy(name='accuracy'),  # Precisión binaria
    keras.metrics.Precision(name='precision'),  # Precisión
    keras.metrics.Recall(name='recall'),  # Sensibilidad
    keras.metrics.AUC(name='auc'),  # Área bajo la curva ROC
]

# Modelo: Entradas, Métricas
def make_model(inputs=['xs', 'ys'], metrics=METRICS):
    input_shape = x_train.shape[1]

    # Cuerpo del modelo

    ## Objeto Sequential.
    model = Sequential()

    ## Capa de entrada: Dimensión de entrada que se ajusta a las columnas de x_train.
    ## Función de activación: "relu" como regla general.
    model.add(Dense(16, activation='relu', input_shape=(input_shape,)))

    ## Nuevas capas:
    ## Agregarlas hasta el modelo se sobreajuste.
    ## Intentaremos luego combatir el sobreajuste con la regularización.
    model.add(Dense(5, activation='relu', input_shape=(16,)))

    ## Capa de salida
    ## La forma de entrada/salida de la capa debe coincidir entre las capas
    model.add(Dense(16, activation='relu', input_shape=(5,)))

    ## Regularización
    ## Las dropout corresponde a una técnica de regularización para prevenir el sobreajuste en modelos de redes neuronales.
    model.add(Dropout(0.5))
    
    # Los valores de salida deben tener la misma dimensión que la función de activación.
    model.add(Dense(1, activation='sigmoid'))

    # Compilación
    # Optimización: Adam (regresión, clasificación) o  mean square (regresión) o gradient (clasificación)
    # Función de pérdida: sparse_cross_entropy
    optimizer = tf.keras.optimizers.Adam()
    model.compile(optimizer=optimizer, loss='sparse_cross_entropy', metrics=metrics)
    
    return model

# Regresión

In [111]:
#Definición del modelo
model = keras.Sequential([
    Dense(64, activation='relu', input_shape=(1,)),  # Capa oculta adicional con 64 unidades y función de activación ReLU
    Dense(32, activation='relu'),                    # Otra capa oculta con 32 unidades y función de activación ReLU
    Dense(1)                                         # Capa de salida con activación lineal (regresión)
])

#Compilación del modelo antes de entrenarlo
model.compile(loss='mean_squared_error',  # Función de pérdida para regresión
              optimizer='adam', # algoritmo de optimización Adam para entrenar tu red neuronal
              metrics=['mae', 'mse'])  # Métricas para evaluar el modelo

# Entrenamiento del modelo
model.fit(x_train, y_train, epochs=100, batch_size=1)

# Evaluación del modelo en el conjunto de prueba
loss, mae, mse = model.evaluate(x_test, y_test)

# Predicción con el modelo entrenado
predictions = model.predict(x_test)

Epoch 1/100
4/4 [==============================] - 1s 5ms/step - loss: 15.7195 - mae: 3.0575 - mse: 15.7195
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 15.3221 - mae: 3.0202 - mse: 15.3221
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 14.8613 - mae: 2.9810 - mse: 14.8613
Epoch 4/100
4/4 [==============================] - 0s 3ms/step - loss: 14.6713 - mae: 2.9613 - mse: 14.6713
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 14.2890 - mae: 2.9282 - mse: 14.2890
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 14.1023 - mae: 2.9075 - mse: 14.1023
Epoch 7/100
4/4 [==============================] - 0s 4ms/step - loss: 13.7013 - mae: 2.8748 - mse: 13.7013
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: 13.3810 - mae: 2.8442 - mse: 13.3810
Epoch 9/100
4/4 [==============================] - 0s 4ms/step - loss: 13.1276 - mae: 2.8200 - mse: 13.1276
Epoch 10/100
4/4 [==========

# Clasificación

In [108]:
# Definición del modelo
model = Sequential([
    Dense(64, activation='relu', input_shape=(1,)),  # Capa oculta adicional con 64 neuronas y función de activación ReLU
    Dense(32, activation='relu'),                    # Otra capa oculta con 32 neuronas y función de activación ReLU
    Dense(1, activation='sigmoid')                    # Capa de salida con función de activación sigmoide (clasificación binaria)
])

model.compile(loss='binary_crossentropy',  # Función de pérdida para clasificación binaria
              optimizer='adam',              # Algoritmo de optimización Adam
              metrics=['accuracy'])           # Métricas para evaluar el modelo

# Entrenamiento del modelo
model.fit(x_train, y_train, epochs=100, batch_size=1)

# Evaluación del modelo en el conjunto de prueba
loss, accuracy = model.evaluate(x_test, y_test)

# Predicción con el modelo entrenado
predictions = model.predict(x_test)

# Redondeamos las predicciones a 0 o 1 para clasificación binaria
rounded_predictions = np.round(predictions)

Epoch 1/100
4/4 [==============================] - 1s 4ms/step - loss: 0.8275 - accuracy: 0.0000e+00
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 0.6922 - accuracy: 0.0000e+00
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 0.4751 - accuracy: 0.0000e+00
Epoch 4/100
4/4 [==============================] - 0s 3ms/step - loss: 0.3945 - accuracy: 0.0000e+00
Epoch 5/100
4/4 [==============================] - 0s 3ms/step - loss: 0.2544 - accuracy: 0.0000e+00
Epoch 6/100
4/4 [==============================] - 0s 3ms/step - loss: 0.1047 - accuracy: 0.0000e+00
Epoch 7/100
4/4 [==============================] - 0s 3ms/step - loss: 0.0216 - accuracy: 0.0000e+00
Epoch 8/100
4/4 [==============================] - 0s 3ms/step - loss: -0.1005 - accuracy: 0.0000e+00
Epoch 9/100
4/4 [==============================] - 0s 3ms/step - loss: -0.1981 - accuracy: 0.0000e+00
Epoch 10/100
4/4 [==============================] - 0s 4ms/step - loss: -0.3067 - accurac

```


La principal diferencia entre los dos fragmentos de código radica en el propósito de los modelos y en la configuración correspondiente utilizada durante la definición, compilación y evaluación del modelo.

Código 1:

Propósito del modelo: Regresión
Arquitectura del modelo: 

El modelo consta de tres capas densas. 
La primera capa tiene 64 unidades y utiliza la función de activación ReLU. 
La segunda capa tiene 32 unidades y también utiliza la función de activación ReLU. 
La última capa es la capa de salida y no se especifica ninguna función de activación, lo que implica que se utiliza una activación lineal.

Código 2:

Propósito del modelo: Clasificación binaria
Arquitectura del modelo: 

El modelo también consta de tres capas densas. 
La primera capa tiene 64 unidades y utiliza la función de activación ReLU. 
La segunda capa tiene 32 unidades y también utiliza la función de activación ReLU. 
La última capa es la capa de salida con la función de activación sigmoide, lo que indica que se trata de un problema de clasificación binaria.

Como capa de salida utiliza una función de activación sigmoide, las predicciones serán probabilidades. Cada valor de predicción será un número en el rango de 0 a 1, que representa la probabilidad estimada de que el ejemplo pertenezca a la clase positiva.

```
